In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
driver = Chrome(executable_path='/Users/chrisjohnson/Downloads/chromedriver')
url = "https://ac-hotels.marriott.com/locations/"
driver.get(url)
time.sleep(30)
viewhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[3]/button')[0]
viewhotels_button.click()
northamhotels_button = driver.find_elements_by_xpath('/html/body/section/pacsys-animation[3]/pacsys-directory/div[2]/pacsys-directory-listings/div/div[5]/a')[0]
northamhotels_button.click()

In [3]:
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
stuff = soup.find_all('a', {'class', 'swell_vertical-xs'})
# stuff
hotel_list = []
for x in stuff:
    hotel = {}
    hotel['name'] = x.text.strip()
    hotel['url'] = x.attrs['href']
    hotel_list.append(hotel)
achotels_df = pd.DataFrame(hotel_list)
achotels_df

,name,url
0,AC Hotel Cape Town Waterfront,https://www.marriott.com/CPTAR
1,AC Hotel Tokyo Ginza,https://www.marriott.com/TYOAR
2,AC Hotel Kuala Lumpur,https://www.marriott.com/KULKA
3,AC Hotel Kuantan,https://www.marriott.com/KUAAR
4,AC Hotel Penang,https://www.marriott.com/PENAR
5,AC Hotel San Jose Escazu,https://www.marriott.com/SJOAR
6,AC Hotel Guatemala City,https://www.marriott.com/GUAAR
7,"AC Hotel Kingston, Jamaica",https://www.marriott.com/KINAR
8,AC Hotel Veracruz,https://www.marriott.com/VERAC
9,"AC Hotel Guadalajara, Mexico",https://www.marriott.com/GDLAC


In [ ]:
%%time

hotel_list = []
counter = 0
for index, row in achotels_df.iterrows():
    name = row['name']
    url = row['url']
    try:
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        stuff = soup.find('div', {'class' : 'l-container l-header-container'})
#             hotel_list = []
        hotel = {}
        try:
            hotel['brand'] = 'AC Hotel'
        except:
            hotel['brand'] = 'AC Hotel'
        try:
            hotel['name'] = name
        except:
            hotel['name'] = name
        try:
            hotel['url'] = url
        except:
            hotel['url'] = url
        try:
            hotel['street'] = stuff.find_all('span')[2].text
        except:
            hotel['street'] = 'N/A'
        try:
            hotel['locality'] = stuff.find_all('span')[4].text
        except:
            hotel['locality'] = 'N/A'        
        try:
            hotel['state'] = stuff.find_all('span')[6].text
        except:
            hotel['state'] = 'N/A'            
        try:
            hotel['postalcode'] = stuff.find_all('span')[7].text
        except:
            hotel['postalcode'] = 'N/A'            
        try:
            hotel['coordinate1'] = stuff.find_all('span')[25].text
        except:
            hotel['coordinate1'] = 'N/A'            
        try:
            hotel['coordinate2'] = stuff.find_all('span')[26].text
        except:
            hotel['coordinate2'] = 'N/A'
        try:
            hotel['coordinate3'] = stuff.find_all('span')[27].text
        except:
            hotel['coordinate3'] = 'N/A'            
        try:
            hotel['coordinate4'] = stuff.find_all('span')[28].text
        except:
            hotel['coordinate4'] = 'N/A'            
        
        desc = soup.find_all('p', {'class' : 'l-margin-top-none l-overflow-hidden t-collapsed-height'})
        try:
            hotel['description'] = str(desc).replace('<p class="l-margin-top-none l-overflow-hidden t-collapsed-height">','').replace('</p>', '')
        except:
            hotel['description'] = 'N/A'
        
        overallratings = stuff = soup.find('div', {'class' : 'l-overall-rating-container'})
        try:
            hotel['overall_rating'] = ratings.find_all('div')[0].text.strip()[:4]
        except:
            hotel['overall_rating'] = 'N/A'
        
        specratings = stuff = soup.find('div', {'class' : 'l-secondary-ratings-container l-clear l-margin-bottom-quarter l-m-margin-bottom-none'})
        try:
            hotel['cleanliness_rating'] = specratings.find_all('div')[2].text.strip()[:4]
        except:
            hotel['cleanliness_rating'] = 'N/A'
        try:
            hotel['service_rating'] = specratings.find_all('div')[7].text.strip()[:4]
        except:
            hotel['service_rating'] = 'N/A'
        
            
        hotel_list.append(hotel)
        print(hotel)
        print()
        print(datetime.now())
        print('='*80)
        print('='*80)
        print()
#         driver.close()
#         driver.switch_to.window(original_window)
#         time.sleep(1)

        counter += 1
        print(f'Collected {counter} hotels.')
    except:
        hotel = {}
        hotel['name'] = name
        hotel['url'] = url
        hotel_list.append(hotel)
        print(f"Number {x} didn't work at {datetime.now()}")
        continue
hotel_df = pd.DataFrame(hotel_list)
hotel_df    

{'brand': 'AC Hotel', 'name': 'AC Hotel Cape Town Waterfront', 'url': 'https://www.marriott.com/CPTAR', 'street': 'Dockrail Road, Foreshore', 'locality': 'Cape Town', 'state': '', 'postalcode': '8001', 'coordinate1': '18.425344', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ A perfectly precise hotel experience awaits at AC Hotel by Marriott Cape Town Waterfront, conveniently located just minutes away from the buzzing Victoria &amp; Alfred Waterfront and just a 25-minute drive from Cape Town International Airport. Our 188 spacious rooms feel stylish, yet efficient and boast unparalleled views over the imposing Table Mountain, Lion's Head and Signal Hill. Savour the heritage of the brand’s Spanish roots with our delectable tapas from fresh and locally sourced ingredients at the AC Lounge &amp; Bar grab a bite at V&amp;A Waterfront. To wind down after a long day, pass by the bar and sample the finest spirits locally crafted. Our bartenders are also t

{'brand': 'AC Hotel', 'name': 'AC Hotel Kingston, Jamaica', 'url': 'https://www.marriott.com/KINAR', 'street': '38-42 Lady Musgrave Road', 'locality': 'Kingston', 'state': '', 'postalcode': '5', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ AC Hotel Kingston, Jamaica is situated on 6 acres of land within the golden triangle and business and entertainment mecca of New Kingston. Your trip will begin virtually the moment your plane touches down, thanks to our unrivaled location just 10 miles from Norman Manley International Airport. Our outdoor pool and bar offer privacy and relaxation complemented by a mouthwatering menu of international cuisine. Our boutique hotel features the AC Lounge, a space of creativity by day and a hub of social buzz by night. During the day, guests will be able to exchange ideas or make the final edits to a presentation in this space designed with the entrepreneurial spirit in mind. At night, the AC Lou

{'brand': 'AC Hotel', 'name': 'AC Hotel Guadalajara Expo, Mexico', 'url': 'https://www.marriott.com/GDLAR', 'street': 'Avenida Lopez Mateos Sur No 2375', 'locality': 'Zapopan', 'state': '', 'postalcode': '45050', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/gdlar-ac-hotel-guadalajara-expo-mexico/?directPageRequest=true', 'coordinate3': '20.648414', 'coordinate4': '-103.403753', 'description': '[ AC Hotel Guadalajara Expo is your new choice when traveling to beautiful Guadalajara, Mexico. Our Hotel is designed with Spanish roots and European soul, offering exceptional experience of service; while staying at our Hotel in Guadalajara you will find a unique combination of quality, modern design, comfort and authenticity. Privileged with an excellent location just 5 minutes distance from Expo Guadalajara, one of the most important Convention Centers in Latin America, easy access to the industrial district (Southern side), near to important growth-bus

{'brand': 'AC Hotel', 'name': 'AC Hotel Ambassadeur Antibes- Juan les Pins', 'url': 'https://www.marriott.com/JLPAR', 'street': '50-52 Chemin des Sables, Antibes', 'locality': 'Juan les Pins', 'state': '', 'postalcode': '06160', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/jlpar-ac-hotel-ambassadeur-antibes-juan-les-pins/?directPageRequest=true', 'coordinate3': '43.56814383', 'coordinate4': '7.11668803999999', 'description': "[ AC Hotel Ambassadeur Antibes- Juan les Pins welcomes you with effortless sophistication and award-winning service. Designed in a thrilling avant-garde style, our hotel is nestled within France's famed French Riviera, mere steps from a pristine private beach. Rooms and suites blend classic modern European design with home-like comforts, and each offers a balcony with views over the hotel pool, the shimmering sea or the Juan les Pins cityscape. Take a dip in the lagoon-style pool in the summer, or enjoy a swim in the indoor

{'brand': 'AC Hotel', 'name': 'AC Hotel Mainz', 'url': 'https://www.marriott.com/FRAMZ', 'street': 'Bahnhofplatz 8', 'locality': 'Mainz', 'state': '', 'postalcode': '55116', 'coordinate1': 'Reviews', 'coordinate2': 'https://www.marriott.com/hotels/maps/travel/framz-ac-hotel-mainz/?directPageRequest=true', 'coordinate3': '50.002249', 'coordinate4': '8.259335', 'description': "[ Connect to the city in a new way at the AC Hotel. Thoughtful service and a modern mindset define us. We've carefully curate the things you really need and edited out what you don't, for an experience that is rich in sophistication, at an unusually smart value. From check-in to check-out, each element of our space has been carefully tailored to alleviate the pains of travel and achieve harmony as you arrive at the AC Hotel Mainz. The best spot to enjoy a fabulous view and to take into the scenery of the area, is the Restaurant and Bar, with its generous window front.Enjoy amenities including WIFI throughout the Ho

...

{'brand': 'AC Hotel', 'name': 'AC Hotel Sants', 'url': 'https://www.marriott.com/BCNSA', 'street': 'Passeig de Sant Antoni, 36-40', 'locality': 'Barcelona', 'state': '', 'postalcode': '08014', 'coordinate1': '2.138364', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ One of the best urban hotels in Barcelona, the AC Hotel Sants, is located near the Sants Train Station, the Fira de Barcelona Exhibition Centre, and the Plaza de España. Our prime location provides easy access to MACBA - the contemporary arts museum, the Sagrada Familia Temple, the airport, and the city centre. The hotel’s location and the quality of its facilities make it an excellent choice for seminars, business meetings, and conventions. Each of the 78 guestrooms has a Wi-Fi Internet connection, and an in-room safe. Indulge in a relaxing moment at the AC Lounge and enjoy the varied menu which includes salads, Spanish tapas, main dishes and delicious desserts. Its magnificent location

{'brand': 'AC Hotel', 'name': 'AC Hotel Guadalajara, Spain', 'url': 'https://www.marriott.com/MADGU', 'street': 'Avda. del Ejército, 6', 'locality': 'Guadalajara', 'state': '', 'postalcode': '19004', 'coordinate1': '-3.172764', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Our Guadalajara hotel features 103 bedrooms that are fully-equipped to meet the needs of our guests with first-class facilities and customer service. The elegant AC Hotel Guadalajara building is located near the the historic town centre, near the beautiful Palacio del Infantado Mansion, constructed in a late Gothic style. Keep in shape at the AC Hotel Guadalajara Fitness Room; open 24 hours a day. Sit back and relax at the AC Lounge, a perfect space for chatting with friends or have an informal meeting and taste a delicious drink with a snack. Guests coming to our town on business or for pleasure will find this urban hotel offers an excellent infrastructure for either. Free Wi-Fi

{'brand': 'AC Hotel', 'name': 'AC Hotel Leon San Antonio', 'url': 'https://www.marriott.com/LENAC', 'street': 'Velazquez,10', 'locality': 'Leon', 'state': '', 'postalcode': '24005', 'coordinate1': '-5.568783', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ AC Hotel León San Antonio is a modern hotel located in the center of the new city, just a few minutes from the Cathedral and the famous Casa de Botines, and next to the city's most complete and modern commercial center. The hotel features mini bar in each guest room, fitness center, a market with everything you need for your trip  and the  AC Lounge, with temporary limited schedule, where you can relax and indulge in a delicious meal. Our León hotel guest rooms are spacious and comfortable, and our event rooms are perfectly equipped for business meetings. Free Wi-Fi access throughout the hotel so that guests can connect to the internet whenever they want. Registration Number for Tourist Accommodat

{'brand': 'AC Hotel', 'name': 'AC Hotel Carlton Madrid', 'url': 'https://www.marriott.com/MADCA', 'street': 'Paseo de la Delicias 26', 'locality': 'Madrid', 'state': '', 'postalcode': '28045', 'coordinate1': 'N/A', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': "[ AC Hotel Carlton Madrid is located in the heart of Madrid, near the Retiro Park, the Botanic Gardens, and the Atocha train and metro station.  It is also just a stone's throw away from the most popular shops and business areas in the city. This 122-bedroom urban hotel features a Wi-Fi Internet connection throughout all communal areas and a bar where you will find a tempting menu of salads and sandwiches throughout the day.  It is minutes away from the Prado, Reina Sofia, and Thyssen-Bornemisza art galleries as well as the Casa Encendida and Caixa Forum cultural centres, ensuring that our guests are within arm's reach of some of the most famous art galleries in Spain. The AC Hotel Carlton Madr

{'brand': 'AC Hotel', 'name': 'AC Hotel Malaga Palacio', 'url': 'https://www.marriott.com/AGPMG', 'street': 'Cortina del Muelle, 1', 'locality': 'Malaga', 'state': '', 'postalcode': '29015', 'coordinate1': '-4.419484', 'coordinate2': 'N/A', 'coordinate3': 'N/A', 'coordinate4': 'N/A', 'description': '[ Our unbeatable location between the Málaga Cathedral and the Paseo del Parque, and our spectacular views of the port, make this hotel the most emblematic in the city. It is near the Picasso museum, the Alcazaba, Castillo de Gibralfaro Castle, and La Malagueta beach. The AC Hotel Málaga Palacio restaurant menu is based on regional cuisine with a variety of dishes from all over the world. In the summer, check out Ático, our bar and restaurant on the 15th floor terrace. With a new urban  spirit we invite you to take a cocktail in our AC Lounge on the first floor with an informal, sophisticate and modern atmosphere in the main center of Málaga City. Our hotel on the Costa del Sol is ideal for

In [ ]:
hotel_df = hotel_df.drop_duplicates(subset='name', ignore_index=True)

In [ ]:
hotel_df

In [8]:
hotel_df.to_csv('achotels_description.csv')